### Importaciones

In [53]:
# Paquetes necesarios para la ejecución del notebook
import win32com.client
import os
from datetime import datetime
import pandas as pd
import shutil
import openpyxl
from openpyxl import load_workbook
from openpyxl.workbook import Workbook
from copy import copy
from openpyxl.utils import get_column_letter, column_index_from_string
import polars as pl
import math
import update_file as uf
import subprocess
import pyautogui
import time
import pygetwindow as gw
from msal import ConfidentialClientApplication
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from PIL import Image
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

### Configuracion

In [54]:
# Constantes usadas en el notebook
MAPI = "MAPI" # Messaging Application Programming Interface
DOT = "."
OUTLOOK = "Outlook"
APPLICATION = "Application"
MAIL_ITEM_CODE = 43

# Diccionarios
outlook_folder_codes = {
    0: 'Calendario',
    1: 'Contactos',
    2: 'Borradores',
    3: 'Diario / Jornal',
    4: 'Notas',
    5: 'Tareas',
    6: 'Bandeja de entrada',
    7: 'Bandeja de salida',
    8: 'Elementos enviados',
    9: 'Elementos eliminados',
    10: 'Bandeja de correo del servidor',
    11: 'Conflictos',
    12: 'Elementos de sincronizacion local',
    13: 'Elementos de sincronizacion (Envio)',
    14: 'Elementos de sincronización (Recibo)',
    15: 'Elementos de sincronización completa',
    16: 'Diario de formularios',
    17: 'Carpeta de búsqueda',
    18: 'Bandeja para reglas cliente',
    19: 'Carpeta de sugerencias de correo',
}
parse_locaciones = {
    '06 AYA EL PEDREGAL': 'El Pedregal',
    '38 AYA ATICO': 'Atico',
    '40 AYA CHALA': 'Chala',
    '88 AYA CAMANA': 'Camana'
}
meta = {
    '06 AYA EL PEDREGAL': {
        2023: {1: 0.7, 2: 0.7, 3: 0.7, 4: 0.7, 5: 0.7, 6: 0.7, 7: 0.7, 8:0.7, 9:0.7, 10:0.7, 11:0.7, 12:0.7},
        2024: {1: 1.2, 2: 1.2, 3: 1.2, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11: 1, 12: 1},
        2025: {1: 0.88, 2: 0.88, 3: 0.88, 4: 0.88, 5: 0.98, 6: 0.88, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0},
    },
    '38 AYA ATICO': {
        2023: {1: 0.7, 2: 0.7, 3: 0.7, 4: 0.7, 5: 0.7, 6: 0.7, 7: 0.7, 8:0.7, 9:0.7, 10:0.7, 11:0.7, 12:0.7},
        2024: {1: 0.4, 2: 0.4, 3: 0.4, 4: 0.48, 5: 0.48, 6: 0.48, 7: 0.48, 8: 0.48, 9: 0.48, 10: 0.48, 11: 0.48, 12: 0.48},
        2025: {1: 0.24, 2: 0.24, 3: 0.24, 4: 0.24, 5: 0.24, 6: 0.24, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0},
    },
    '40 AYA CHALA': {
        2023: {1: 0.7, 2: 0.7, 3: 0.7, 4: 0.7, 5: .7, 6: 0.7, 7: 0.7, 8:0.7, 9:0.7, 10:0.7, 11:0.7, 12:0.7},
        2024: {1: 0.5, 2: 0.5, 3: 0.5, 4: 0.60, 5: 0.60, 6: 0.60, 7: 0.60, 8: 0.60, 9: 0.60, 10: 0.60, 11: 0.60, 12: 0.60},
        2025: {1: 0.31, 2: 0.31, 3: 0.31, 4: 0.31, 5: 0.31, 6: 0.31, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0},
    },
    '88 AYA CAMANA': {
        2023: {1: 0.7, 2: 0.7, 3: 0.7, 4: 0.7, 5: 0.7, 6: 0.7, 7: 0.7, 8:0.7, 9:0.7, 10:0.7, 11:0.7, 12:0.7},
        2024: {1: 0.85, 2: 0.85, 3: 0.85, 4: 0.85, 5: 0.85, 6: 0.85, 7: 0.85, 8: 0.85, 9: 0.85, 10: 0.85, 11: 0.85, 12: 0.85},
        2025: {1: 0.57, 2: 0.57, 3: 0.57, 4: 0.57, 5: 0.57, 6: 0.57, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0},
    }
}
vendedores = {
    '06 AYA EL PEDREGAL': {
        '0000001013': 'AYALA CCAHUANA EDMUNDO',
        '0000001006': 'BARRAZA REVILLA FREDY',
        '0000001000': 'CARDENAS CHOQUECAJIA WHITNEY AWARD',
        '0000001001': 'CHARA ROJAS GISELA SOLEDAD',
        '0000001007': 'CHINO QUISPE JORGE LUIS',
        '0000006014': 'EDWAR QUISPE CHINO',
        '0000006009': 'GALLARDO HUAYLLA RENATO LEUTERIO',
        '0000006003': 'GALLEGOS NUÑEZ JUAN FREDY',
        '0000005999': 'GESTOR VIRTUAL',
        '0000001010': 'HUANCA MAMANI JACKSON',
        '0000006006': 'HUAYCHO TORRES ULBER',
        '0000006005': 'HUAYCHO TORRES ULBER',
        '0000001012': 'HUAYLLAZO HUACCHA CRISTHIAN PAOLO',
        '0000006012': 'NELSON RAUL CONZA CHARCA',
        '0000001002': 'PACHECO CONDORI ANDRES OSCAR',
        '0000001011': 'QUISPE CHINO WILIAN YURI',
        '0000001004': 'QUISPE HUAYLLA MARISOL S.',
        '0000006013': 'RONALD GONZALO HUILLCA MAMANI',
        '0000001003': 'TACO XESSPE KELY SOFIA',
        '0000001005': 'VALDERRAMA ELLIS JESSICA A.',
        '0000001008': 'VEND NVA RUTA 10',
        '0000001009': 'YUCRA JIMENEZ ANA LUZ',
        '0000001014': 'CHARA ROJAS GISELA SOLEDAD',
        '0000001015': 'BARRAZA REVILLA FREDY',
        '0000001019': 'BATALLANOS SANCA RODRIGO LEOPOLDO',
        '0000001016': 'PARI PUCHO FREDY OSWALDO',
        '0000001017': 'COAGUILLA MAMANI JOSE ALBERTO',
        '0000001018': 'ORTEGA MAMANI JORGE LUIS'
    },
    '38 AYA ATICO': {
        '0000001001': 'ALANYA RAMIREZ FERNANDO JOSUE',
        '0000005999': 'GESTOR VIRTUAL',
        '0000001000': 'SAUL ANDRES VIÑA VIZCARDO'
    },
    '40 AYA CHALA': {
        '0000001004': 'CANALES AGUILAR HILBERTO',
        '0000001007': 'CHIPANA JURADO JHORS EDUARDO',
        '0000006010': 'DIONICIO DANIEL HUARCAYA SALAZAR',
        '0000005999': 'GESTOR VIRTUAL',
        '0000006009': 'GLOBER FELIPE JARA MAQUER',
        '0000006005': 'GONZALES CHURA MAGNO ALFREDO',
        '0000001000': 'HERRERA TAPIA EDWIN DONATO',
        '0000006002': 'JARA CARAZAS RODOLFO JOSUE.L',
        '0000006007': 'JUAN CARLOS ARIAS BENITES',
        '0000001003': 'MAMANI TINTAYA KRISTHOFER',
        '0000001006': 'MAMANI TINTAYA KRISTHOFER',
        '0000001005': 'QUISE CCAPA EVER',
        '0000001002': 'RODRIGUEZ JOSE ANTONIO',
        '0000001001': 'VENDEDOR RT M1'        
    },
    '88 AYA CAMANA': {
        '0000001007': 'AUCAHUAQUI REVILLA DANIEL HITLER',
        '0000001011': 'CARAZAS REZA LUIS ALBERTO',
        '0000001003': 'CONDORCHOA SIERRA NEMECIO',
        '0000006001': 'CONDORCHOA SIERRA NEMESIO JESUS',
        '0000006002': 'DE LA CRUZ CALCINA ELAR',
        '0000006010': 'EDILBERTO RAMIREZ LARICO',
        '0000005999': 'GESTOR VIRTUAL',
        '0000001002': 'HUAMANI RODRIGUEZ YONATAN ANYIMZAN',
        '0000006011': 'JAIME CHAVEZ CONDORI',
        '0000001005': 'LLERENA DE LA CRUZ RICARDO SNEIDER',
        '0000001004': 'MEDINA VELASQUEZ JAVIER ENRIQUE',
        '0000006014': 'MOISES RICHARD CONDORCHOA SIERRA',
        '0000001008': 'MOLLO YUPANQUI JOSE OMAR',
        '0000001001': 'NO APLICA VENDEDOR',
        '0000006008': 'QUISPE CCACHUCO FREDY',
        '0000001010': 'RAMOS MAMANI RUBEN',
        '0000006016': 'RENATO ELEUTERIO GALLARADO HUAYLLA',
        '0000001006': 'SACARI CHOQUEHUANCA WILSON',
        '0000001000': 'SALAZAR HUAMANI SAUL ANTONIO',
        '0000001012': 'VEND RT X1',
        '0000001009': 'VIZCARDO BUSTAMANTE ALBERTH FRANCK'
    }
}

# Listas
locaciones = ['06 AYA EL PEDREGAL', '38 AYA ATICO', '40 AYA CHALA', '88 AYA CAMANA']

# Variables de configuracion
root_address = r'C:\Informacion\rechazos' # Direcccion de carpeta raiz
test_address = r'\prueba'
backup_address = r'\backup'
transportista = {
    "mail_subject": "Reporte de ordenes de carga diario",         # Nombre del asunto de correo
    "local_file_name": "Cf_programadas_por_transportista.csv",    # Nombre del archivo local
    "local_file_address": "",                                     # Direccion del archivo local
    "mail_file_address": "",                                      # Direccion del archivo del correo
    "mail_sheet_name": "Guias",
    "mail_received_time": "",
    "date": "Fecha",
    "date_format": "dd\/mm\/yyyy",
}
transportista_resumen = {
    "mail_subject": "Reporte de ordenes de carga diario",                 # Nombre del asunto de correo
    "local_file_name": "Cf_programadas_por_transportista_resumen.csv",    # Nombre del archivo local
    "local_file_address": "",                                             # Direccion del archivo local
    "date": "Fecha",
    "date_format": "dd\/mm\/yyyy",
}
ruta = {
    "mail_subject": "Venta perdida diaria por cliente y ruta - acum mes",   # Nombre del asunto de correo
    "local_file_name": "Cf_rech_por_ruta.csv",                              # Nombre del archivo local
    "local_file_address": "",                                               # Direccion del archivo local
    "mail_file_address": "",                                                # Direccion del archivo del correo
    "mail_sheet_name": "Motivos_VP (clte)",
    "mail_received_time": "",
    "date": "Día",
    "date_format": "d\/m\/yy",
}

<>:151: SyntaxWarning: invalid escape sequence '\/'
<>:158: SyntaxWarning: invalid escape sequence '\/'
<>:168: SyntaxWarning: invalid escape sequence '\/'
<>:151: SyntaxWarning: invalid escape sequence '\/'
<>:158: SyntaxWarning: invalid escape sequence '\/'
<>:168: SyntaxWarning: invalid escape sequence '\/'
C:\Users\AYACDA23\AppData\Local\Temp\ipykernel_14060\361112561.py:151: SyntaxWarning: invalid escape sequence '\/'
  "date_format": "dd\/mm\/yyyy",
C:\Users\AYACDA23\AppData\Local\Temp\ipykernel_14060\361112561.py:158: SyntaxWarning: invalid escape sequence '\/'
  "date_format": "dd\/mm\/yyyy",
C:\Users\AYACDA23\AppData\Local\Temp\ipykernel_14060\361112561.py:168: SyntaxWarning: invalid escape sequence '\/'
  "date_format": "d\/m\/yy",


### Obtener correos de outlook

In [55]:
# Conectar a Outlook
#outlook_folder_code = int(input(f'{" ".join(["(" + str(key) + ": " + value + ")" for key, value in outlook_folder_codes.items()])}'))
outlook = win32com.client.Dispatch(OUTLOOK+DOT+APPLICATION).GetNamespace(MAPI)

outlook_folder = outlook.GetDefaultFolder(6)
print("Tipo de folder: ", outlook_folder)

Tipo de folder:  Bandeja de entrada


### Guardar archivo de outlook

In [56]:
# Buscar el correo más reciente con archivo Excel
mails = outlook_folder.Items

# Ordenar por fecha descendente
mails.Sort("[ReceivedTime]", True) # (mails) Es un objeto lista

''' MAIL PROPERTIES
    mail.Subject
    mail.ReceivedTime
    mail.SenderName
    mail.SenderEmailAddress
    mail.To
    mail.CC
    mail.Body
    mail.Attachments.Count
    mail.CreationTime
    mail.LastModificationTime
    mail.EntryID
'''

# Descargar archivos Excel de los correos
def download_excel_file(mails, mail_subject, mail_sheet_name):
    for mail in mails: # Recorrer todos los correos
        if mail.Class != MAIL_ITEM_CODE: # Asegurar que el correo sea un MailItem (otros pueden ser calendario, alertas, etc)
            continue
        
        if (mail_subject.lower() in mail.Subject.lower()) & (mail.Attachments.Count > 0):
            for attachment in mail.Attachments:
                if attachment.FileName.endswith((".xlsx", ".xls")):
                    file_name = attachment.FileName
                    file_address = os.path.join(root_address+test_address, file_name)
                    attachment.SaveAsFile(file_address)

                    # Validar si hay que revisar una hoja
                    if mail_sheet_name:
                        try:
                            # Cargar solo la hoja específica
                            df = pd.read_excel(file_address, sheet_name=mail_sheet_name, header=None)
                            df = uf.clean_mail_file(df)

                            # Eliminar filas que están completamente vacías
                            df_clean = df.dropna(how='all')

                            # Si la hoja esta completamente vacía, eliminar el archivo
                            if df.empty:
                                print(f"Archivo '{file_name}' con fecha {mail.ReceivedTime} está vacío, eliminando...")
                                os.remove(file_address)
                                continue
                            
                            # Si no hay datos pero si encabezados, eliminar el archivo
                            elif df_clean.shape[0] == 0:
                                print(f"Archivo '{file_name}' con fecha {mail.ReceivedTime} está vacío, eliminando...")
                                os.remove(file_address)
                                continue

                        except Exception as e:
                            print(f"Error leyendo hoja '{mail_sheet_name}' en {file_name}: {e}")
                            continue

                    # Si pasó las validaciones, devolver                    
                    return file_address, mail.ReceivedTime

transportista['mail_file_address'], transportista['mail_received_time'] = download_excel_file(mails, transportista['mail_subject'], transportista['mail_sheet_name'])
ruta['mail_file_address'], ruta['mail_received_time'] = download_excel_file(mails, ruta['mail_subject'], ruta['mail_sheet_name'])

c:\Users\AYACDA23\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Archivo 'Reporte de Guias de Distribución por FechaV3_53_9035756698100090800.xlsx' con fecha 2025-06-02 08:38:34+00:00 está vacío, eliminando...


c:\Users\AYACDA23\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\AYACDA23\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Archivo 'Venta perdida x Cliente y ruta diaria_3136_3712873636512649727.xlsx' con fecha 2025-06-02 08:39:00+00:00 está vacío, eliminando...


c:\Users\AYACDA23\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


### Establecer configuracion de documentos

In [57]:
transportista['local_file_address'] = os.path.join(root_address+test_address, transportista['local_file_name'])
transportista_resumen['local_file_address'] = os.path.join(root_address+test_address, transportista_resumen['local_file_name'])
ruta['local_file_address'] = os.path.join(root_address+test_address, ruta['local_file_name'])

# Configuracion de documentos
def imprimir_diccionario(nombre, diccionario):
    print(f'\n{nombre.upper()}:')
    for clave, valor in diccionario.items():
        print(f'{clave}: {valor}')

imprimir_diccionario('Transportista', transportista)
imprimir_diccionario('Transportista Resumen', transportista_resumen)
imprimir_diccionario('Ruta', ruta)


TRANSPORTISTA:
mail_subject: Reporte de ordenes de carga diario
local_file_name: Cf_programadas_por_transportista.csv
local_file_address: C:\Informacion\rechazos\prueba\Cf_programadas_por_transportista.csv
mail_file_address: C:\Informacion\rechazos\prueba\Reporte de Guias de Distribución por FechaV3_54_563490388953043607.xlsx
mail_sheet_name: Guias
mail_received_time: 2025-06-01 08:24:56+00:00
date: Fecha
date_format: dd\/mm\/yyyy

TRANSPORTISTA RESUMEN:
mail_subject: Reporte de ordenes de carga diario
local_file_name: Cf_programadas_por_transportista_resumen.csv
local_file_address: C:\Informacion\rechazos\prueba\Cf_programadas_por_transportista_resumen.csv
date: Fecha
date_format: dd\/mm\/yyyy

RUTA:
mail_subject: Venta perdida diaria por cliente y ruta - acum mes
local_file_name: Cf_rech_por_ruta.csv
local_file_address: C:\Informacion\rechazos\prueba\Cf_rech_por_ruta.csv
mail_file_address: C:\Informacion\rechazos\prueba\Venta perdida x Cliente y ruta diaria_1128_6544381617250754382.

### Actualizar Transportista y Ruta

In [58]:
def update_local_file(document):
    # Rutas de archivo
    mail_file_address = document['mail_file_address']
    mail_sheet_name = document['mail_sheet_name']
    local_file_address = os.path.join(root_address+test_address, document['local_file_name'])

    # Leer datos del archivo local
    df_local = pd.read_csv(local_file_address, sep=';')
    df_local = uf.delete_unnamed_columns(df_local)

    # Leer datos del archivo de correo
    df_mail = pd.read_excel(mail_file_address, sheet_name=mail_sheet_name, header=None)
    df_mail = uf.clean_mail_file(df_mail)
    df_mail = uf.filter_mail_file_locations(df_mail, locaciones)

    # Filtrar por fecha
    df_mail = uf.filter_mail_file_dates(document, df_mail, df_local)

    if len(df_mail) > 0:        
        # Configuracion solo para el archivo de ruta
        df_mail = uf.customized_ruta_mail_file(df_mail, vendedores)

        # Asegurar los mismos tipos de datos (str, int, ...)
        df_mail.columns = df_local.columns
        df_mail = df_mail.astype(df_local.dtypes.to_dict())

        # Convertir la fecha a string, incluso si los valores son datetime dentro de "object"
        df_mail[document['date']] = df_mail[document['date']].apply(lambda x: x.strftime('%d/%m/%Y') if isinstance(x, pd.Timestamp) or isinstance(x, datetime) else str(x))
        #print(document['date'].apply(type).value_counts())

        # Convertir a Polars
        df_local = pl.from_pandas(df_local)
        df_mail = pl.from_pandas(df_mail)

        if df_mail.schema != df_local.schema:
            print("⚠️ Los esquemas no coinciden")

        # print(f'Local: {df_local.schema}')
        print(df_local.shape)
        # print(f'Mail: {df_mail.schema}')
        print(df_mail.shape)

        # Combinar los datos (sin eliminar duplicados)
        df_updated = pl.concat([df_local, df_mail])

        # print(f'Updated: {df_mail.schema}')
        print(df_updated.shape)

        return df_updated
    
    else:
        print(f"⚠️ No hay datos nuevos en el archivo de correo '{mail_file_address}' para la hoja '{mail_sheet_name}'.")

        return pl.from_pandas(df_local)

transportista_updated = update_local_file(transportista)
ruta_updated = update_local_file(ruta)

c:\Users\AYACDA23\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


⚠️ No hay datos nuevos en el archivo de correo 'C:\Informacion\rechazos\prueba\Reporte de Guias de Distribución por FechaV3_54_563490388953043607.xlsx' para la hoja 'Guias'.


c:\Users\AYACDA23\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


⚠️ No hay datos nuevos en el archivo de correo 'C:\Informacion\rechazos\prueba\Venta perdida x Cliente y ruta diaria_1128_6544381617250754382.xlsx' para la hoja 'Motivos_VP (clte)'.


c:\Informacion\rechazos\Analisis\automatizacion_reporte_rechazos\update_file.py:31: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_local_copy[document['date']+"2"] = pd.to_datetime(df_local_copy[document['date']], dayfirst=True, errors='coerce')


### Actualizar Transportista Resumen

In [59]:
# Función para asignar la meta desde el diccionario
def obtener_meta(loc, año, mes):
    try:
        return meta[loc][año][mes]
    except KeyError:
        return 0.0

In [60]:
# Resumen CF de transportista

df = transportista_updated

# Asegurar tipos
df = df.with_columns([
    pl.col("Fecha").str.strptime(pl.Date, "%d/%m/%Y", strict=False),
    pl.col("Carga Pvta CF").cast(pl.Float64),
    pl.col("Rechazo CF").cast(pl.Float64)
])

# Agregar columnas de año y mes para usar en la función meta
df = df.with_columns([
    pl.col("Fecha").dt.year().alias("Año"),
    pl.col("Fecha").dt.month().alias("Mes")
])

# Agrupar por Fecha y Locación
transportista_resumen_updated = df.group_by(["Fecha", "Locación", "Año", "Mes"]).agg([
    pl.sum("Carga Pvta CF").alias("Total CF Programandas"),
    pl.sum("Rechazo CF").alias("Total CF Rechazadas")
])

# Aplicar la función de meta
transportista_resumen_updated = transportista_resumen_updated.with_columns([
    pl.struct(["Locación", "Año", "Mes"]).map_elements(
        lambda x: obtener_meta(x["Locación"], x["Año"], x["Mes"]),
        return_dtype=pl.Float64
    ).alias("Meta")
])

# Calcular columnas adicionales
transportista_resumen_updated = transportista_resumen_updated.with_columns([
    pl.col("Total CF Programandas").round(2),
    pl.col("Total CF Rechazadas").round(2),
    (pl.col("Meta") / 2).alias("Alerta"),
    (pl.col("Meta") * 2).alias("End"),
    (
        (pl.col("Total CF Rechazadas") / pl.col("Total CF Programandas"))
        .fill_nan(0)
        .fill_null(0)
    ).alias("% Rechazo")
])

# Formato final (opcional: redondear y convertir % a string)
transportista_resumen_updated = transportista_resumen_updated.select([
    "Fecha",
    "Locación",
    "Total CF Programandas",
    "Total CF Rechazadas",
    pl.col("Meta").map_elements(lambda x: f"{math.ceil(x * 100) / 100:.2f}%", return_dtype=pl.Utf8).alias("Meta"),
    pl.col("Alerta").map_elements(lambda x: f"{math.ceil(x * 100) / 100:.2f}%", return_dtype=pl.Utf8).alias("Alerta"),
    pl.col("End").map_elements(lambda x: f"{math.ceil(x * 100) / 100:.2f}%", return_dtype=pl.Utf8).alias("End"),
    pl.col("% Rechazo").map_elements(lambda x: f"{x:.2%}", return_dtype=pl.Utf8).alias("% Rechazo")
])

#print(f'Transportista resumen: {transportista_resumen_updated.schema}')
# print(transportista_resumen_updated.shape)

### Update Files

In [61]:
uf.save_local_file_changes(transportista_updated, transportista)
uf.save_local_file_changes(ruta_updated, ruta)
uf.save_local_file_changes(transportista_resumen_updated, transportista_resumen)

### Continuar con envio de reporte

Ahora se actualizara y subira manualmente power bi

In [62]:
continue_with_automatizaion = ""
yes = 'si'

while continue_with_automatizaion.lower() != yes:
    continue_with_automatizaion = str(input("¿Desea continuar con la automatización? (si/no): "))

### Enviar reporte por correo

Configuracion

In [64]:
# Configuracion usadas para el uso de selenium y chromedriver
RECHAZOS_PAGE = 'https://app.powerbi.com/groups/me/reports/73309ec0-8d79-4111-ac74-acee01ed5375/ReportSection487ba5c07a522bf0fc8c'
DETALLES_PAGE = 'https://app.powerbi.com/groups/me/reports/73309ec0-8d79-4111-ac74-acee01ed5375/ReportSection3f59cb816b6f52a24171'
METAS_Y_RESUMEN_PAGE = 'https://app.powerbi.com/groups/me/reports/73309ec0-8d79-4111-ac74-acee01ed5375/ReportSectionf1e1f415e4cba9cc4035'
REPORTES = ["% CF Rechazadas", "CF Rech. por Motivo", "CF Rech. por Transportista", "Historico % Rechazo por día"]

In [ ]:
# ==== CONFIGURACIÓN DE SELENIUM ====
options = Options()
options.add_argument("--start-maximized")
options.add_argument("--user-data-dir=C:\\Users\\AYACDA23\\AppData\\Local\\Google\\Chrome\\User Data\\Profile 6")

driver = webdriver.Chrome(options=options)

try:
    print("[*] Abriendo Power BI...")
    driver.get(RECHAZOS_PAGE)

    print("[*] Esperando a que cargue el gráfico...\n")
    wait = WebDriverWait(driver, 30)
    time.sleep(20)

    for reporte in REPORTES:
        print(f'[*] Buscando el gráfico {reporte}')        

        try:
            # Buscar todos los elementos con aria-label dentro de visual-container
            candidatos = driver.find_elements(By.XPATH, f"//visual-container//*[contains(@aria-label, '{reporte}')]")

            if not candidatos:
                raise Exception(f"❌ No se encontró ningún gráfico con el texto: '{reporte}'")

            for cand in candidatos:
                # Subimos al ancestro visual-container
                try:
                    contenedor = cand.find_element(By.XPATH, "./ancestor::visual-container")
                    grafico = contenedor.find_element(By.CLASS_NAME, "visualContainer")
                    break
                except:
                    continue
            else:
                raise Exception(f"❌ No se encontró visualContainerHost para: '{reporte}'")

            # Captura
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", grafico)
            time.sleep(2)
            grafico.screenshot(f'grafico_{reporte}.png')
            print(f"[✓] Imagen guardada como 'grafico_{reporte}.png'\n")

        except Exception as e:
            print(f"[X] Error: no se pudo encontrar o capturar '{reporte}' → {e}")

finally:
    driver.quit()

[*] Abriendo Power BI...
[*] Esperando a que cargue el gráfico...
[*] Buscando el gráfico % CF Rechazadas
[✓] Imagen guardada como 'grafico_% CF Rechazadas.png'

[*] Buscando el gráfico CF Rech. por Motivo
[✓] Imagen guardada como 'grafico_CF Rech. por Motivo.png'

[*] Buscando el gráfico CF Rech. por Transportista
[✓] Imagen guardada como 'grafico_CF Rech. por Transportista.png'

[*] Buscando el gráfico Historico % Rechazo por día
[✓] Imagen guardada como 'grafico_Historico % Rechazo por día.png'

